# Building Dangerous Live-Coding Playgrounds with Jupyter Widgets

![http://i294.photobucket.com/albums/mm110/eaglewebtech/2014-03-24/dangerous-and-scary-playgrounds_0_zpsf7a875d6.jpg](http://i.imgur.com/aIAiWxQ.jpg)

I want to be able to build any kind of "playground" application, where each keystroke in any number of source documents updates an output document. Some examples:
- [Online YAML Parser](http://yaml-online-parser.appspot.com/)
- [JSON-LD Playground](http://json-ld.org/playground/)
- [PEG Grammar](http://pegjs.org/online)
- [js2coffee](http://js2.coffee/)
- [babel REPL](https://babeljs.io/repl/)


## Widgets and traitlets
[traitlets](https://github.com/ipython/traitlets)offer an [observable implementation](https://en.wikipedia.org/wiki/Observer_pattern) that makes the state of a single value observable, to which your code can react... in Python.

[Widgets](https://github.com/ipython/ipywidgets), made up of traitlets, make the state of the data of a named collection of traitelts to be updated, and reacted to, simultaneously on the kernel (backend) and in the notebook (front-end). This lets us use both backend modules and front-end modules to achieve transformations.

## Ingredients
- a live text source: [CodeMirror](https://codemirror.net/) is already available in the notebook, so let's widgetize that
- a transformer: while traitlets has `link`, this doesn't let us supply arbitrary transformations

Let's get going!

In [9]:
import traitlets
import ipywidgets as widgets

In [10]:
# Convenience methods, as the traitlets API is a little verbose
# for the widget base case.
Unicode = lambda *x: traitlets.CUnicode(*x).tag(sync=True)
Bool = lambda *x: traitlets.CBool(*x).tag(sync=True)
Any = lambda *x: traitlets.Any(*x).tag(sync=True)

## The CodeMirror Widget: Javascript
Writing and distributing widgets is a bit of a pain right now: however, we can (ab)use the live Jupyter Notebook to just ram some named modules right into the runtime environment.

In [11]:
%%javascript
requirejs.undef("widget_codemirror");

define("widget_codemirror",
[
    "underscore",
    "jquery",
    "components/codemirror/lib/codemirror",
    "nbextensions/widgets/widgets/js/widget",
],
function(_, $, CodeMirror, widget){
    var CodeMirrorView = widget.DOMWidgetView.extend({
        render: function(){
            _.bindAll(this, "_init", "_cm_changed", "_mode_loaded");

            this.displayed.then(this._init);
            
            // wire up magic functions
            _.map(
                ["value", "theme", "mode", "description"],
                function(name){
                    this.listenTo(
                        this.model,
                        "change:"+name,
                        this["_" + name + "_changed"]
                    );
                }, this);
        },
        _init: function(){
            this.$description = $("<div/>").appendTo(this.$el);
            
            this._cm = new CodeMirror($("<div/>").appendTo(this.$el)[0], {
                value: this.m("value")
            });
            this._cm.on("change", this._cm_changed);
            this._theme_changed();
            this._mode_changed();
            this._description_changed();
        },
        _cm_changed: function(){
            this.m("value", this.cmv());
            this.touch();
        },
                                       
        // model listeners
        _value_changed: function(){
            var value = this.m("value");
            this._cm.hasFocus() || value === this.cmv() || this.cmv(value);
        },
        _theme_changed: function(){
            var theme = this.m("theme"),
                href="/static/components/codemirror/theme/" + theme + ".css",
                style = $('link[href$="' + theme +'"]');
            if(theme && !style.length){
                $("<link/>", {rel: "stylesheet", href: href})
                    .appendTo($("head"));
            }
            this._cm.setOption("theme", theme);
        },
        
        _description_changed: function(){
            this.$description.text(this.m("description"));
        },
        _mode_changed: function(){
            var mode = this.m("mode");
            mode && require([
                ["components/codemirror/mode",  mode, mode].join("/")
            ], this._mode_loaded)
        },
        
        _mode_loaded: function(){
            this._cm.setOption("mode", this.m("mode"));
        },
        
        cmv: function(_new){
            var old = this._cm.getValue();
            if(arguments.length){
                old === _new || this._cm.setValue(_new);
                return this;
            }
            return old;
        },
        // d3-style (g|s)etter
        m: function(name, val){
            if(arguments.length == 2){
                this.model.set(name, val);
                return this;
            }
            return this.model.get(name);
        }
    });
    
    var CodeMirrorModel = widget.WidgetModel.extend({});
    
    return {
        CodeMirrorView: CodeMirrorView,
        CodeMirrorModel: CodeMirrorModel 
    };
});

<IPython.core.display.Javascript object>

## The CodeMirror Widget: Python
With the JavaScript defined, we can start building stuff!

In [12]:
class CodeMirror(widgets.DOMWidget):
    _view_module = Unicode("widget_codemirror")
    _view_name = Unicode("CodeMirrorView")
    _model_module = Unicode("widget_codemirror")
    _model_name = Unicode("CodeMirrorModel")
    
    value = Unicode()
    description = Unicode()
    
    mode = Unicode()
    theme = Unicode("monokai")
    
    def __init__(self, value="", *args, **kwargs):
        super(CodeMirror, self).__init__(value=value, *args, **kwargs)

In [13]:
cm = CodeMirror("print('hello world')", description="Yay, code!", mode="python", theme="monokai")
cm

# The Pipe Widget
The playground pattern is usually
```
(some stuff, some other stuff) → (yet more stuff)
```
Let's generalize this into a widget.

In [14]:
class Pipe(widgets.Widget):
    # a tuple of (widget, "trait")s... or, if setting "value", just (widget)
    sources = traitlets.Tuple([])
    # a single (widget, "trait")
    dest = traitlets.Tuple()
    # a place to put the last error
    error = traitlets.Any()

    def __init__(self, fn, *args, **kwargs):
        super(Pipe, self).__init__(*args, **kwargs)
        # this doesn't need to be a traitlet, probably
        self.fn = fn
    
    def _update(self):
        [src[0].observe(self._src_changed) for src in self.sources]

    def _sources_changed(self, old, new):
        self._update()

    def _dest_changed(self, old, new):
        self._update()
    
    def _src_changed(self, change):
        args = [
            getattr(src[0], src[1] if len(src) == 2 else "value")
            for src in self.sources
        ]
        try:
            setattr(self.dest[0],
                    self.dest[1] if len(self.dest) == 2 else "value",
                    self.fn(*args))
        except Exception as err:
            self.error = err

## First, some sliders
No widget demo would be complete without some sliders. Here is a very simple example of adding two numbers to update a third.

In [15]:
import operator

x, y, z = [widgets.FloatSlider(value=1, description=it) for it in "xyz"]
p = Pipe(operator.mul, sources=[[x], [y]], dest=[z])
widgets.VBox(children=[x,y,z])

# A YAML Playground
YAML is a great language for writing data quickly.

In [16]:
import yaml
import json

class YamlPlayground(widgets.FlexBox):
    json = traitlets.Any({})

    def __init__(self, *args, **kwargs):
        self._yaml = CodeMirror("x: 1", description="YAML",  mode="yaml", width="50%")
        self._json = CodeMirror(description="JSON", mode="javascript", width="50%")
        
        # transform to YAML
        Pipe(lambda x: json.dumps(yaml.safe_load(x), indent=2),
             sources=[[self._yaml]], dest=[self._json])
        
        kwargs.update(
            children=[self._yaml, self._json],
            orientation="horizontal"
        )

        super(YamlPlayground, self).__init__(*args, **kwargs)
YamlPlayground()

# [wip] JSON-LD Playground
We use the new text editor, the pipe and the standard layout widgets to make a re-creation of the JSON-LD Playground. First, we'll need some documents to start with.

In [27]:
doc = """{
  "@context": {
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "name" : "rdfs:label"
  },
  "name": "Jane Doe"
}"""

context = """{
  "@context": {
    "rdfs": "http://www.w3.org/2000/01/rdf-schema#",
    "name" : "rdfs:label"
  }
}"""

In [28]:
from pyld import jsonld

class JSONLDPlayground(widgets.FlexBox):
    doc = traitlets.Any({})
    context = traitlets.Any({})
    compacted = traitlets.Any({})

    def __init__(self, *args, **kwargs):
        self._doc = CodeMirror(doc, description="Document", width="33%")
        self._context = CodeMirror(value=context, description="Context", width="33%")
        self._compacted = CodeMirror(value="{}", description="Compacted", width="33%")
        
        # transform to JSON
        Pipe(json.loads, sources=[[self._doc]], dest=[self, "doc"])
        Pipe(json.loads, sources=[[self._context]], dest=[self, "context"])
        Pipe(lambda x: json.dumps(x, indent=2), sources=[[self, "compacted"]], dest=[self._compacted])
        
        # finally, we can compact the JSON-LD
        Pipe(jsonld.compact,
             sources=[[self, "doc"], [self, "context"]],
             dest=[self, "compacted"])

        kwargs.update(children=[self._doc, self._context, self._compacted],
                      orientation="horizontal")

        super(JSONLDPlayground, self).__init__(*args, **kwargs)
pg = JSONLDPlayground()
pg

## To be continued...